In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import sys
import pickle
import warnings
import shutil
import os
import pandas as pd
import re
from moviepy.video.io.VideoFileClip import VideoFileClip
import json

%matplotlib inline
warnings.filterwarnings('ignore')

In [2]:
# import imageio
# imageio.plugins.ffmpeg.download()

In [3]:
def plot_boundaries(img_path, x1, y1, x2, y2):
    img = cv2.imread(img_path)
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2BGRA)
    plt.figure(figsize=(25,28))
    plt.imshow(gray)
    plt.show()

In [4]:
# plot_boundaries('data/frames/1.1.1_frame_01523.jpg',2695,435,2743,514)

Labels: Each line contains 10+ columns, separated by spaces. The definition of these columns are:

    Track ID. All rows with the same ID belong to the same person for 180 frames. Then the person gets a new idea for the next 180 frames. We will soon release an update to make the IDs consistant.
    xmin. The top left x-coordinate of the bounding box.
    ymin. The top left y-coordinate of the bounding box.
    xmax. The bottom right x-coordinate of the bounding box.
    ymax. The bottom right y-coordinate of the bounding box.
    frame. The frame that this annotation represents.
    lost. If 1, the annotation is outside of the view screen.
    occluded. If 1, the annotation is occluded.
    generated. If 1, the annotation was automatically interpolated.
    label. The label for this annotation, enclosed in quotation marks. This field is always “Person”.
    (+) actions. Each column after this is an action.


In [5]:
# ffmpeg -i ../1.1.1.mov -start_number 0 frame_%05d.jpg
def split_video(video_file, image_name_prefix, output_path):
	return subprocess.check_output('ffmpeg -i ' + os.path.abspath(video_file) + ' '+ image_name_prefix +'%d.jpg', shell=True, cwd=os.path.join(destination_path, 'JPEGImages'))\

def convert_video_to_sequences(video_file_path, video_file_name, output_path):
    image_list = None
    if video_file_path is not None and output_path is not None:
        video = VideoFileClip(video_file_path)
        image_list = video.write_images_sequence('{0}/{1}_frame_%05d.jpg'.format(output_path, video_file_name), fps=None, verbose=True, withmask=True, progress_bar=True)
    return image_list

In [6]:
# Create json file for a frame
def create_frame_level_annotations(annotation_file, video_file_name, output_path):
    # map {frame : list(annotations)}
    sep = ' '
    columns = ['track_id', 'xmin', 'ymin', 'xmax', 'ymax','frame','lost','occluded','generated','label','action']
    cols_req = ['track_id', 'xmin', 'ymin', 'xmax', 'ymax','lost','occluded','generated','label','action']
    annotation_df = pd.read_csv(annotation_file, sep=sep , names=columns, header=None)
    
    frames = annotation_df.frame.unique()
    for frame in frames:
        frame_df = annotation_df[np.logical_and(annotation_df.frame == frame, annotation_df.lost == 0)][cols_req]
        with open('{0}/{1}_frame_{2:05d}.json'.format(output_path, video_file_name, frame), 'w') as fp:
            json.dump(list(frame_df.T.to_dict().values()), fp)
    return frames

In [7]:
# create_frame_level_annotations('data/Labels/SingleActionLabels/3840x2160/1.1.1.txt', '1.1.1','data/annotations')
# convert_video_to_sequences('data/1.1.1.mov', '1.1.1', 'data/frames')

In [10]:
# create folder for frames and annotation : frame_name and annotation will be same
# create metadata csv file with index which has information about frames and annotation <index, image_file_name, ann_file_name>

root_video_folder = '/home/vijin/iith/project/data/okutama-action-drone-data/videos'
root_annotation_folder = '/home/vijin/iith/project/data/okutama-action-drone-data/video-annotations'

frame_output_folder = '/home/vijin/iith/project/data/okutama-action-drone-data/frames'
ann_output_folder = '/home/vijin/iith/project/data/okutama-action-drone-data/annotations'
root_folder = '/home/vijin/iith/project/data/okutama-action-drone-data'

ann_ext = '.txt'

meta_data = pd.DataFrame(columns=['id', 'img_file', 'ann_file'])
file_counter = 0

for x in os.listdir(root_video_folder):
    video_file_name = x[0:x.rindex('.')]
    video_file_path = '{0}/{1}'.format(root_video_folder, x)
    ann_file = '{0}/{1}{2}'.format(root_annotation_folder, video_file_name, ann_ext)
    
    print('Processing {0}...'.format(x))
    image_list = convert_video_to_sequences(video_file_path, video_file_name, frame_output_folder)

    print('Creating frame level annotations...')
    frames = create_frame_level_annotations(ann_file, video_file_name, ann_output_folder)
    
    meta_data_tup = []
    for index, file_path in enumerate(image_list):
        frame_number_search = re.search('.*_frame_(\d+).jpg', file_path[file_path.rindex('/')+1 :], re.IGNORECASE)
        if frame_number_search:
            if int(frame_number_search.group(1)) in frames:
                meta_data_tup.append((file_counter+index, file_path[file_path.rindex('/')+1 :],
                      '{0}.json'.format(file_path[file_path.rindex('/')+1 :file_path.rindex('.')])))
        
#     meta_data_tup = [(file_counter+index, file_path[file_path.rindex('/')+1 :],
#                       '{0}.json'.format(file_path[file_path.rindex('/')+1 :file_path.rindex('.')])) 
#                      for index, file_path in enumerate(image_list)]
    meta_data = meta_data.append(pd.DataFrame(meta_data_tup, columns=['id', 'img_file', 'ann_file']), ignore_index=True)
    
    
    file_counter = file_counter + len(image_list)
    
meta_data.to_csv('{0}/metadata.csv'.format(root_folder), index=False)
print('Finished processing....')

Processing 2.1.6.mp4...
[MoviePy] Writing frames /home/vijin/iith/project/data/okutama-action-drone-data/frames/2.1.6_frame_%05d.jpg.


100%|██████████| 3/3 [00:00<00:00,  3.60it/s]
4it [00:01,  3.71it/s]                       
5it [00:01,  3.78it/s]
6it [00:01,  3.77it/s]
7it [00:01,  3.68it/s]
8it [00:02,  3.67it/s]
9it [00:02,  3.78it/s]
10it [00:02,  3.80it/s]
11it [00:02,  3.88it/s]
12it [00:03,  3.86it/s]
13it [00:03,  3.90it/s]
14it [00:03,  3.94it/s]
15it [00:03,  3.92it/s]
16it [00:04,  3.91it/s]
17it [00:04,  3.93it/s]
18it [00:04,  3.88it/s]
19it [00:04,  3.86it/s]
20it [00:05,  3.79it/s]
21it [00:05,  3.85it/s]
22it [00:05,  3.88it/s]
23it [00:05,  3.93it/s]
24it [00:06,  3.97it/s]
25it [00:06,  3.97it/s]
26it [00:06,  3.98it/s]
27it [00:06,  4.02it/s]
28it [00:07,  4.03it/s]
29it [00:07,  3.92it/s]
30it [00:07,  3.94it/s]
31it [00:07,  3.96it/s]
32it [00:08,  3.91it/s]
33it [00:08,  3.95it/s]
2519it [10:28,  3.95it/s]

[MoviePy]: Done writing frames /home/vijin/iith/project/data/okutama-action-drone-data/frames/2.1.6_frame_%05d.jpg.



Creating frame level annotations...
Processing 2.2.11.mp4...
[MoviePy] Writing frames /home/vijin/iith/project/data/okutama-action-drone-data/frames/2.2.11_frame_%05d.jpg.

1263it [04:47,  4.78it/s]                    

...

[MoviePy]: Done writing frames /home/vijin/iith/project/data/okutama-action-drone-data/frames/1.1.4_frame_%05d.jpg.



Creating frame level annotations...
Processing 2.1.7.mp4...
[MoviePy] Writing frames /home/vijin/iith/project/data/okutama-action-drone-data/frames/2.1.7_frame_%05d.jpg.

2511it [10:21,  3.98it/s]                    

[MoviePy]: Done writing frames /home/vijin/iith/project/data/okutama-action-drone-data/frames/2.1.7_frame_%05d.jpg.



Creating frame level annotations...
Finished processing....
